In [ ]:
import cv2
import numpy as np

In [ ]:
# reading image and converting it to grayscale
img = cv2.imread('test_images/test_image_1.png', cv2.IMREAD_GRAYSCALE)
  
# # converting image into grayscale image
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Converting to grayscale allows for easy detection of objects, but still different colours give different gradients of gray, so it is necessary to threshold them to find contours.
Thresholding is a very popular segmentation technique, used for separating an object considered as a foreground from its background.

The function used is cv2.threshold. First argument is the source image, which should be a grayscale image. Second argument is the threshold value which is used to classify the pixel values. Third argument is the maxVal which represents the value to be given if pixel value is more than the threshold value. OpenCV provides different styles of thresholding and it is decided by the fourth parameter of the function.

cv2.THRESH_BINARY: If pixel intensity is greater than the set threshold, value set to 255, else set to 0 (black).

Syntax: cv2.threshold(source, thresholdValue, maxVal, thresholdingTechnique) 
Parameters: 
-> source: Input Image array (must be in Grayscale). 
-> thresholdValue:(all pixel values above this value will be changed to white(255), whereas other values to be changed to black(0)).
-> maxVal: Maximum value that can be assigned to a pixel. 
-> thresholdingTechnique: The type of thresholding to be applied. 

cv2.threshold returns two values, only one is needed for shape detection

In [ ]:
_, threshold = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)

Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity.

There are three arguments in cv2.findContours() function, first one is source image, second is contour retrieval mode, third is contour approximation method. And it outputs the contours and hierarchy.

The mode cv2.RETR_TREE finds all the promising contour lines and reconstructs a full hierarchy of nested contours. The method cv2.CHAIN_APPROX_SIMPLE returns only the endpoints that are necessary for drawing the contour line. It removes all redundant points and compresses the contour, thereby saving memory (as compared to cv2.CHAIN_APPROX_NONE, which stores all points)

In [ ]:
contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours
First argument is source and destination image, second argument is the contours which should be passed as a Python list, third argument is index of contours (useful when drawing individual contour. To draw all contours, pass -1) and remaining arguments are color, thickness etc.
We'll count the vertices as the intersecting points of the contours to determine the shape.

This may have errorenous contours near the vertices, so we use approximation.
cv2.approxPolyDP(input_curve, epsilon, closed)
It approximates a contour shape to another shape with less number of vertices depending upon the precision we specify.
Second argument is called epsilon, which is maximum distance from contour to approximated contour. It is an accuracy parameter. A wise selection of epsilon is needed to get the correct output. The third argument is True, which indicates that the shape is closed.
    cv2.arcLength() is used to calculate the perimeter of the contour(cnt) and takes 1% (0.01) of this length for cv2.approxPolyDP as epsilon. If the second argument is True then it considers the contour to be closed. 

In [ ]:
#to loop over the contours
for cnt in contours:
    
    # using drawContours() function (non-approximated)
    #cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
    
    #approximating
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
    
    cv2.drawContours(img, [approx], 0, (0), 5)
  
    # # finding center point of shape
    # M = cv2.moments(cnt)
    # if M['m00'] != 0.0:
    #     x = int(M['m10']/M['m00'])
    #     y = int(M['m01']/M['m00'])
  
    # # putting shape name at center of each shape
    # if len(approx) == 3:
    #     cv2.putText(img, 'Triangle', (x, y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    # elif len(approx) == 4:
    #     cv2.putText(img, 'Square', (x, y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    # elif len(approx) == 5:
    #     cv2.putText(img, 'Pentagon', (x, y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    # elif len(approx) == 6:
    #     cv2.putText(img, 'Hexagon', (x, y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    # else:
    #     cv2.putText(img, 'circle', (x, y),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

In [ ]:
cv2.imshow("shapes", img)
cv2.imshow("Thresholded", threshold)
cv2.waitKey(0)
cv2.destroyAllWindows()